##### Usecase:Get sales data by orderstatus
###### Solutions are given for following problem statements
###### Problem statement-1:Get sales data for each orderstatus
###### Problem statement-2:Get sales data for each orderstatus perday
###### Problem statement-3:Get total orders for each order status permonth
###### Problem statement-4:Get total orders for each order status peryear
###### Problem statement-5:Get monthly sales data for each order status for first quarter of 2014
###### Problem statement-6:Get monthly sales data for last quarter of 2013 with neither 'COMPLETE' nor 'CLOSED' orders
###### Problem statement-7:Get monthly sales data for last quarter of 2013 and first quarter of 2014  with 'CANCELED' orders
###### Problem statement-8:Get monthly sales data for 'PENDING' orders
###### Problem statement-9:Get monthly sales data for 'PROCESSING' OR ON_HOLD orders
###### Problem statement-10:Get monthly sales data with  'COMPLETE' AND 'CLOSED' orders

In [2]:
from pyspark.sql import SQLContext, Row
from pyspark.sql import HiveContext
from pyspark.sql.functions import round,sum,avg,count
hiveContext = HiveContext(sc)
sqlContext.sql("set spark.sql.shuffle.partitions=10")

Out[11]: DataFrame[key: string, value: string]

In [3]:
hiveContext.sql("create database retail_db")
hiveContext.sql("use retail_db")

In [4]:
hiveContext.sql("create table orders(order_id int,order_date string,order_customer_id int, order_status string) row format delimited fields terminated by ','")
hiveContext.sql("create table order_items(order_item_id int, order_item_order_id int, order_item_product_id  int, order_item_quantity int,order_item_subtotal float,order_item_product_price float) row format delimited fields terminated by ','")

In [5]:
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Orders.txt' into table orders")
hiveContext.sql("load data local inpath 'dbfs:/FileStore/tables/retaildbtext/Order_Items.txt' into table order_items")

In [6]:

hiveContext.sql("show tables").show()

+---------+-----------+-----------+
 database| tableName|isTemporary|
+---------+-----------+-----------+
retail_db| categories| false|
retail_db| customers| false|
retail_db|departments| false|
retail_db|order_items| false|
retail_db| orders| false|
retail_db| products| false|
+---------+-----------+-----------+

In [7]:
hiveContext.sql("select * from orders").show(5,False)

+--------+-----------------------+-----------------+---------------+
order_id|order_date |order_customer_id|order_status |
+--------+-----------------------+-----------------+---------------+
1 |2013-07-25 00:00:00.000|11599 |CLOSED |
2 |2013-07-25 00:00:00.000|256 |PENDING_PAYMENT|
3 |2013-07-25 00:00:00.000|12111 |COMPLETE |
4 |2013-07-25 00:00:00.000|8827 |CLOSED |
5 |2013-07-25 00:00:00.000|11318 |COMPLETE |
+--------+-----------------------+-----------------+---------------+
only showing top 5 rows

In [8]:
hiveContext.sql("select * from order_items").show(5,False)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

##### Problem statement-1:Get sales data for each orderstatus

In [10]:
SalesDataPerorderPerstatus = hiveContext.sql('''select otl.order_status,round(sum(oitl.order_item_subtotal),2) as Totalrevenue_perstatus, 
                                                count(1) as Numberoforder_perstatus,sum(oitl.order_item_quantity) as Totalquantity_perstatus 
                                                from Orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                                group by otl.order_status 
                                                order by otl.order_status''')
SalesDataPerorderPerstatus.coalesce(1).write.csv("/FileStore/tables/SalesDataPerorderPerstatus ", compression="none", header ='true')
SalesDataPerorderPerstatus.show(20,False)

+---------------+----------------------+-----------------------+-----------------------+
order_status |Totalrevenue_perstatus|Numberoforder_perstatus|Totalquantity_perstatus|
+---------------+----------------------+-----------------------+-----------------------+
CANCELED |696030.99 |3519 |7702.0 |
CLOSED |3736048.79 |18668 |40510.0 |
COMPLETE |1.127693369E7 |56740 |123873.0 |
ON_HOLD |1864731.24 |9373 |20414.0 |
PAYMENT_REVIEW |357841.45 |1797 |4013.0 |
PENDING |3851881.28 |19291 |42260.0 |
PENDING_PAYMENT|7581671.05 |38031 |82935.0 |
PROCESSING |4190636.76 |20901 |45622.0 |
SUSPECTED_FRAUD|766844.68 |3878 |8429.0 |
+---------------+----------------------+-----------------------+-----------------------+

##### Problem statement-2:Get sales data for each orderstatus perday

In [12]:
SalesDataPerstatusPerday = sqlContext.sql('''select substring(otl.order_date,0,10) as orderdate, otl.order_status, 
                                             round(sum(oitl.order_item_subtotal),2) as Totalrevenue_perstatus,
                                             count(1) as Numberoforder_perstatus, oitl.order_item_quantity,sum(oitl.order_item_quantity) as Totalquantity_perstatus
                                             from Orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                             group by orderdate,otl.order_status,oitl.order_item_quantity 
                                             order by orderdate,otl.order_status''')
SalesDataPerstatusPerday.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusPerday", compression="none", header ='true')
SalesDataPerstatusPerday.show(20,False)

+----------+--------------+----------------------+-----------------------+-------------------+-----------------------+
orderdate |order_status |Totalrevenue_perstatus|Numberoforder_perstatus|order_item_quantity|Totalquantity_perstatus|
+----------+--------------+----------------------+-----------------------+-------------------+-----------------------+
2013-07-25|CANCELED |429.97 |2 |1 |2.0 |
2013-07-25|CLOSED |7229.6 |32 |1 |32.0 |
2013-07-25|CLOSED |1007.8 |5 |4 |20.0 |
2013-07-25|CLOSED |599.88 |4 |3 |12.0 |
2013-07-25|CLOSED |829.88 |7 |2 |14.0 |
2013-07-25|CLOSED |1849.75 |6 |5 |30.0 |
2013-07-25|COMPLETE |4909.2 |15 |5 |75.0 |
2013-07-25|COMPLETE |1220.76 |9 |3 |27.0 |
2013-07-25|COMPLETE |2639.28 |12 |4 |48.0 |
2013-07-25|COMPLETE |1263.72 |12 |2 |24.0 |
2013-07-25|COMPLETE |9997.36 |48 |1 |48.0 |
2013-07-25|ON_HOLD |200.0 |1 |4 |4.0 |
2013-07-25|ON_HOLD |1149.92 |7 |1 |7.0 |
2013-07-25|ON_HOLD |250.0 |1 |5 |5.0 |
2013-07-25|ON_HOLD |299.92 |3 |2 |6.0 |
2013-07-25|PAYMENT_REVIEW|559.92 |2 |4 |8.0 |
2013-07-25|PAYMENT_REVIEW|749.85 |2 |5 |10.0 |
2013-07-25|PAYMENT_REVIEW|109.97 |2 |1 |2.0 |
2013-07-25|PENDING |367.92 |2 |4 |8.0 |
2013-07-25|PENDING |489.92 |5 |2 |10.0 |
+----------+--------------+----------------------+-----------------------+-------------------+-----------------------+
only showing top 20 rows

##### Problem statement-3:Get total orders for each order status permonth

In [14]:
SalesDataPerstatusPermonth = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales_perstatus,
                                               count(1) as Numberoforder_perstatus,sum(oitl.order_item_quantity) as Totalquantity_perstatus
                                               from Orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                               group by ordermonth,otl.order_status 
                                               order by ordermonth,otl.order_status''')
SalesDataPerstatusPermonth.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusPermonth", compression="none", header ='true')
SalesDataPerstatusPermonth.show(20,False)

+----------+---------------+--------------------+-----------------------+-----------------------+
ordermonth|order_status |Totalsales_perstatus|Numberoforder_perstatus|Totalquantity_perstatus|
+----------+---------------+--------------------+-----------------------+-----------------------+
2013-07 |CANCELED |9882.41 |58 |136.0 |
2013-07 |CLOSED |78107.44 |382 |833.0 |
2013-07 |COMPLETE |255358.01 |1291 |2880.0 |
2013-07 |ON_HOLD |37594.59 |199 |435.0 |
2013-07 |PAYMENT_REVIEW |7747.87 |43 |103.0 |
2013-07 |PENDING |86945.94 |419 |953.0 |
2013-07 |PENDING_PAYMENT|175973.91 |862 |1824.0 |
2013-07 |PROCESSING |97341.74 |493 |1097.0 |
2013-07 |SUSPECTED_FRAUD|15830.28 |74 |160.0 |
2013-08 |CANCELED |59422.67 |318 |730.0 |
2013-08 |CLOSED |314631.12 |1593 |3472.0 |
2013-08 |COMPLETE |907197.78 |4585 |10083.0 |
2013-08 |ON_HOLD |155155.39 |768 |1658.0 |
2013-08 |PAYMENT_REVIEW |25696.76 |139 |295.0 |
2013-08 |PENDING |345071.17 |1736 |3809.0 |
2013-08 |PENDING_PAYMENT|604406.69 |2998 |6627.0 |
2013-08 |PROCESSING |349112.78 |1786 |3889.0 |
2013-08 |SUSPECTED_FRAUD|67964.34 |345 |748.0 |
2013-09 |CANCELED |67973.94 |324 |687.0 |
2013-09 |CLOSED |338312.49 |1706 |3647.0 |
+----------+---------------+--------------------+-----------------------+-----------------------+
only showing top 20 rows

##### Problem statement-4:Get total orders for each order status peryear

In [16]:
SalesDataPerstatusPeryear = sqlContext.sql('''select substring(otl.order_date,0,4) as orderyear, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales_perstatus, 
                                               count(1) as Numberoforder_perstatus,sum(oitl.order_item_quantity) as Totalquantity_perstatus
                                               from Orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                               group by orderyear,otl.order_status
                                               order by orderyear,otl.order_status''')
SalesDataPerstatusPeryear.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusPeryear", compression="none", header ='true')
SalesDataPerstatusPeryear.show(20,False)

+---------+---------------+--------------------+-----------------------+-----------------------+
orderyear|order_status |Totalsales_perstatus|Numberoforder_perstatus|Totalquantity_perstatus|
+---------+---------------+--------------------+-----------------------+-----------------------+
2013 |CANCELED |314083.11 |1603 |3513.0 |
2013 |CLOSED |1713090.0 |8574 |18588.0 |
2013 |COMPLETE |4973802.0 |24951 |54616.0 |
2013 |ON_HOLD |829421.2 |4126 |9075.0 |
2013 |PAYMENT_REVIEW |143455.81 |742 |1594.0 |
2013 |PENDING |1715070.59 |8596 |18972.0 |
2013 |PENDING_PAYMENT|3386687.17 |17076 |36995.0 |
2013 |PROCESSING |1839024.49 |9224 |20281.0 |
2013 |SUSPECTED_FRAUD|339554.7 |1741 |3775.0 |
2014 |CANCELED |381947.88 |1916 |4189.0 |
2014 |CLOSED |2022958.79 |10094 |21922.0 |
2014 |COMPLETE |6303131.69 |31789 |69257.0 |
2014 |ON_HOLD |1035310.04 |5247 |11339.0 |
2014 |PAYMENT_REVIEW |214385.64 |1055 |2419.0 |
2014 |PENDING |2136810.69 |10695 |23288.0 |
2014 |PENDING_PAYMENT|4194983.88 |20955 |45940.0 |
2014 |PROCESSING |2351612.27 |11677 |25341.0 |
2014 |SUSPECTED_FRAUD|427289.98 |2137 |4654.0 |
+---------+---------------+--------------------+-----------------------+-----------------------+

##### Problem statement-5:Get monthly sales data for each order status for first quarter of 2014

In [18]:
SalesDataPerstatusfirstquarter = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales_perstatus,
                                               count(1) as Numberoforder_perstatus,sum(oitl.order_item_quantity) as Totalquantity_perstatus
                                               from Orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                               where CAST(otl.order_date AS DATE)>='2014-01-01' AND CAST(otl.order_date AS DATE)<='2014-03-31'
                                               group by ordermonth,otl.order_status 
                                               order by ordermonth,otl.order_status''')
SalesDataPerstatusfirstquarter.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusfirstquarter", compression="none", header ='true')
SalesDataPerstatusfirstquarter.show(30,False)

+----------+---------------+--------------------+-----------------------+-----------------------+
ordermonth|order_status |Totalsales_perstatus|Numberoforder_perstatus|Totalquantity_perstatus|
+----------+---------------+--------------------+-----------------------+-----------------------+
2014-01 |CANCELED |53612.83 |263 |591.0 |
2014-01 |CLOSED |298236.03 |1492 |3247.0 |
2014-01 |COMPLETE |931985.71 |4706 |9958.0 |
2014-01 |ON_HOLD |181414.96 |894 |1913.0 |
2014-01 |PAYMENT_REVIEW |35816.65 |185 |404.0 |
2014-01 |PENDING |342057.66 |1699 |3818.0 |
2014-01 |PENDING_PAYMENT|670633.08 |3376 |7324.0 |
2014-01 |PROCESSING |346943.34 |1706 |3760.0 |
2014-01 |SUSPECTED_FRAUD|63746.75 |345 |754.0 |
2014-02 |CANCELED |65825.08 |318 |667.0 |
2014-02 |CLOSED |302636.05 |1547 |3251.0 |
2014-02 |COMPLETE |915134.04 |4667 |10251.0 |
2014-02 |ON_HOLD |148648.08 |754 |1642.0 |
2014-02 |PAYMENT_REVIEW |29113.01 |139 |329.0 |
2014-02 |PENDING |316896.05 |1576 |3486.0 |
2014-02 |PENDING_PAYMENT|612843.93 |3046 |6728.0 |
2014-02 |PROCESSING |327870.18 |1696 |3625.0 |
2014-02 |SUSPECTED_FRAUD|59697.24 |291 |659.0 |
2014-03 |CANCELED |57485.89 |292 |634.0 |
2014-03 |CLOSED |301673.83 |1514 |3344.0 |
2014-03 |COMPLETE |969677.14 |4880 |10642.0 |
2014-03 |ON_HOLD |162486.62 |805 |1687.0 |
2014-03 |PAYMENT_REVIEW |38871.57 |191 |431.0 |
2014-03 |PENDING |305423.93 |1513 |3343.0 |
2014-03 |PENDING_PAYMENT|601666.67 |3032 |6541.0 |
2014-03 |PROCESSING |353374.53 |1756 |3821.0 |
2014-03 |SUSPECTED_FRAUD|71832.03 |367 |778.0 |
+----------+---------------+--------------------+-----------------------+-----------------------+

##### Problem statement-6:Get monthly sales data for last quarter of 2013 with neither 'COMPLETE' nor 'CLOSED' orders

In [20]:
SalesDataPerstatuslastquarter = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales_perstatus,
                                               count(1) as Numberoforder_perstatus,sum(oitl.order_item_quantity) as Totalquantity_perstatus
                                               from Orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                               where CAST(otl.order_date AS DATE)>='2013-10-01' AND CAST(otl.order_date AS DATE)<='2013-12-31'
                                               group by ordermonth,otl.order_status 
                                               having otl.order_status!='COMPLETE' AND otl.order_status!='CLOSED'
                                               order by ordermonth,otl.order_status''')
SalesDataPerstatuslastquarter.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatuslastquarter", compression="none", header ='true')
SalesDataPerstatuslastquarter.show(30,False)

+----------+---------------+--------------------+-----------------------+-----------------------+
ordermonth|order_status |Totalsales_perstatus|Numberoforder_perstatus|Totalquantity_perstatus|
+----------+---------------+--------------------+-----------------------+-----------------------+
2013-10 |CANCELED |51024.94 |256 |560.0 |
2013-10 |ON_HOLD |144669.59 |735 |1616.0 |
2013-10 |PAYMENT_REVIEW |22879.35 |121 |278.0 |
2013-10 |PENDING |273922.89 |1397 |3102.0 |
2013-10 |PENDING_PAYMENT|566179.7 |2857 |6107.0 |
2013-10 |PROCESSING |340758.38 |1703 |3814.0 |
2013-10 |SUSPECTED_FRAUD|53478.84 |277 |581.0 |
2013-11 |CANCELED |62812.76 |318 |702.0 |
2013-11 |ON_HOLD |181362.63 |912 |1990.0 |
2013-11 |PAYMENT_REVIEW |27480.01 |147 |312.0 |
2013-11 |PENDING |364237.38 |1822 |4022.0 |
2013-11 |PENDING_PAYMENT|721418.18 |3664 |7890.0 |
2013-11 |PROCESSING |358435.03 |1773 |3840.0 |
2013-11 |SUSPECTED_FRAUD|72974.71 |375 |824.0 |
2013-12 |CANCELED |62966.39 |329 |698.0 |
2013-12 |ON_HOLD |147652.14 |716 |1591.0 |
2013-12 |PAYMENT_REVIEW |26757.83 |139 |298.0 |
2013-12 |PENDING |322284.06 |1635 |3527.0 |
2013-12 |PENDING_PAYMENT|682300.31 |3483 |7566.0 |
2013-12 |PROCESSING |353135.69 |1735 |3798.0 |
2013-12 |SUSPECTED_FRAUD|60148.25 |314 |661.0 |
+----------+---------------+--------------------+-----------------------+-----------------------+

###### Problem statement-7:Get monthly sales data for last quarter of 2013 and first quarter of 2014  with 'CANCELED' orders

In [22]:
SalesDatacanceledordersquarter = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales,round(avg(oitl.order_item_subtotal),2) as Averagesales,
                                               count(1) as Numberoforders,sum(oitl.order_item_quantity) as Totalquantity
                                               from Orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                               where CAST(otl.order_date AS DATE)>='2013-10-01' AND CAST(otl.order_date AS DATE)<='2014-03-31'
                                               group by ordermonth,otl.order_status 
                                               having otl.order_status =='CANCELED'
                                               order by ordermonth,otl.order_status''')
SalesDatacanceledordersquarter.coalesce(1).write.csv("/FileStore/tables/SalesDatacanceledordersquarter", compression="none", header ='true')
SalesDatacanceledordersquarter.show(30,False)

+----------+------------+----------+------------+--------------+-------------+
ordermonth|order_status|Totalsales|Averagesales|Numberoforders|Totalquantity|
+----------+------------+----------+------------+--------------+-------------+
2013-10 |CANCELED |51024.94 |199.32 |256 |560.0 |
2013-11 |CANCELED |62812.76 |197.52 |318 |702.0 |
2013-12 |CANCELED |62966.39 |191.39 |329 |698.0 |
2014-01 |CANCELED |53612.83 |203.85 |263 |591.0 |
2014-02 |CANCELED |65825.08 |207.0 |318 |667.0 |
2014-03 |CANCELED |57485.89 |196.87 |292 |634.0 |
+----------+------------+----------+------------+--------------+-------------+

###### Problem statement-8:Get monthly sales data for 'PENDING' orders

In [24]:
SalesDatapendingorder = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales,round(avg(oitl.order_item_subtotal),2) as Averagesales,
                                               count(1) as Numberoforders,sum(oitl.order_item_quantity) as Totalquantity
                                               from Orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                               group by ordermonth,otl.order_status 
                                               having otl.order_status LIKE'%PEND%'
                                               order by ordermonth,otl.order_status''')
SalesDatapendingorder.coalesce(1).write.csv("/FileStore/tables/SalesDatapendingorder", compression="none", header ='true')
SalesDatapendingorder.show(30,False)

+----------+---------------+----------+------------+--------------+-------------+
ordermonth|order_status |Totalsales|Averagesales|Numberoforders|Totalquantity|
+----------+---------------+----------+------------+--------------+-------------+
2013-07 |PENDING |86945.94 |207.51 |419 |953.0 |
2013-07 |PENDING_PAYMENT|175973.91 |204.15 |862 |1824.0 |
2013-08 |PENDING |345071.17 |198.77 |1736 |3809.0 |
2013-08 |PENDING_PAYMENT|604406.69 |201.6 |2998 |6627.0 |
2013-09 |PENDING |322609.15 |203.28 |1587 |3559.0 |
2013-09 |PENDING_PAYMENT|636408.38 |198.13 |3212 |6981.0 |
2013-10 |PENDING |273922.89 |196.08 |1397 |3102.0 |
2013-10 |PENDING_PAYMENT|566179.7 |198.17 |2857 |6107.0 |
2013-11 |PENDING |364237.38 |199.91 |1822 |4022.0 |
2013-11 |PENDING_PAYMENT|721418.18 |196.89 |3664 |7890.0 |
2013-12 |PENDING |322284.06 |197.12 |1635 |3527.0 |
2013-12 |PENDING_PAYMENT|682300.31 |195.89 |3483 |7566.0 |
2014-01 |PENDING |342057.66 |201.33 |1699 |3818.0 |
2014-01 |PENDING_PAYMENT|670633.08 |198.65 |3376 |7324.0 |
2014-02 |PENDING |316896.05 |201.08 |1576 |3486.0 |
2014-02 |PENDING_PAYMENT|612843.93 |201.2 |3046 |6728.0 |
2014-03 |PENDING |305423.93 |201.87 |1513 |3343.0 |
2014-03 |PENDING_PAYMENT|601666.67 |198.44 |3032 |6541.0 |
2014-04 |PENDING |287068.61 |195.42 |1469 |3133.0 |
2014-04 |PENDING_PAYMENT|630317.75 |199.09 |3166 |6912.0 |
2014-05 |PENDING |296962.02 |198.64 |1495 |3286.0 |
2014-05 |PENDING_PAYMENT|612500.01 |202.28 |3028 |6720.0 |
2014-06 |PENDING |313181.43 |200.5 |1562 |3308.0 |
2014-06 |PENDING_PAYMENT|584000.49 |203.34 |2872 |6439.0 |
2014-07 |PENDING |275220.99 |199.29 |1381 |2914.0 |
2014-07 |PENDING_PAYMENT|483021.95 |198.37 |2435 |5276.0 |
+----------+---------------+----------+------------+--------------+-------------+

###### Problem statement-9:Get monthly sales data for 'PROCESSING' or ON_HOLD orders

In [26]:
SalesDataprocessingcanceledorders = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales,round(avg(oitl.order_item_subtotal),2) as Averagesales,
                                               count(1) as Numberoforders,sum(oitl.order_item_quantity) as Totalquantity
                                               from Orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                               group by ordermonth,otl.order_status 
                                               having otl.order_status LIKE'%PROCESS%'OR otl.order_status LIKE'%HOLD%'
                                               order by ordermonth,otl.order_status''')
SalesDataprocessingcanceledorders.coalesce(1).write.csv("/FileStore/tables/SalesDataprocessingcanceledorders", compression="none", header ='true')
SalesDataprocessingcanceledorders.show(30,False)

+----------+------------+----------+------------+--------------+-------------+
ordermonth|order_status|Totalsales|Averagesales|Numberoforders|Totalquantity|
+----------+------------+----------+------------+--------------+-------------+
2013-07 |ON_HOLD |37594.59 |188.92 |199 |435.0 |
2013-07 |PROCESSING |97341.74 |197.45 |493 |1097.0 |
2013-08 |ON_HOLD |155155.39 |202.03 |768 |1658.0 |
2013-08 |PROCESSING |349112.78 |195.47 |1786 |3889.0 |
2013-09 |ON_HOLD |162986.86 |204.76 |796 |1785.0 |
2013-09 |PROCESSING |340240.87 |196.22 |1734 |3843.0 |
2013-10 |ON_HOLD |144669.59 |196.83 |735 |1616.0 |
2013-10 |PROCESSING |340758.38 |200.09 |1703 |3814.0 |
2013-11 |ON_HOLD |181362.63 |198.86 |912 |1990.0 |
2013-11 |PROCESSING |358435.03 |202.16 |1773 |3840.0 |
2013-12 |ON_HOLD |147652.14 |206.22 |716 |1591.0 |
2013-12 |PROCESSING |353135.69 |203.54 |1735 |3798.0 |
2014-01 |ON_HOLD |181414.96 |202.93 |894 |1913.0 |
2014-01 |PROCESSING |346943.34 |203.37 |1706 |3760.0 |
2014-02 |ON_HOLD |148648.08 |197.15 |754 |1642.0 |
2014-02 |PROCESSING |327870.18 |193.32 |1696 |3625.0 |
2014-03 |ON_HOLD |162486.62 |201.85 |805 |1687.0 |
2014-03 |PROCESSING |353374.53 |201.24 |1756 |3821.0 |
2014-04 |ON_HOLD |154928.83 |192.7 |804 |1767.0 |
2014-04 |PROCESSING |366710.8 |198.76 |1845 |3954.0 |
2014-05 |ON_HOLD |150186.95 |197.87 |759 |1618.0 |
2014-05 |PROCESSING |324532.18 |203.98 |1591 |3447.0 |
2014-06 |ON_HOLD |131820.94 |191.04 |690 |1511.0 |
2014-06 |PROCESSING |347167.07 |209.26 |1659 |3691.0 |
2014-07 |ON_HOLD |105823.66 |195.61 |541 |1201.0 |
2014-07 |PROCESSING |285014.17 |200.15 |1424 |3043.0 |
+----------+------------+----------+------------+--------------+-------------+

###### Problem statement-10:Get monthly sales data with  'COMPLETE' AND 'CLOSED' orders

In [28]:
SalesDatacompleteandclosedorders = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth, otl.order_status,
                                               round(sum(oitl.order_item_subtotal),2) as Totalsales_perstatus,
                                               count(1) as Numberoforder_perstatus,sum(oitl.order_item_quantity) as Totalquantity_perstatus
                                               from Orders otl join order_items oitl on otl.order_id=oitl.order_item_order_id 
                                               group by ordermonth,otl.order_status 
                                               having otl.order_status=='COMPLETE' OR otl.order_status=='CLOSED'
                                               order by ordermonth,otl.order_status''')
SalesDatacompleteandclosedorders.coalesce(1).write.csv("/FileStore/tables/SalesDatacompleteandclosedorders", compression="none", header ='true')
SalesDatacompleteandclosedorders.show(30,False)

+----------+------------+--------------------+-----------------------+-----------------------+
ordermonth|order_status|Totalsales_perstatus|Numberoforder_perstatus|Totalquantity_perstatus|
+----------+------------+--------------------+-----------------------+-----------------------+
2013-07 |CLOSED |78107.44 |382 |833.0 |
2013-07 |COMPLETE |255358.01 |1291 |2880.0 |
2013-08 |CLOSED |314631.12 |1593 |3472.0 |
2013-08 |COMPLETE |907197.78 |4585 |10083.0 |
2013-09 |CLOSED |338312.49 |1706 |3647.0 |
2013-09 |COMPLETE |963943.31 |4826 |10536.0 |
2013-10 |CLOSED |294414.06 |1468 |3184.0 |
2013-10 |COMPLETE |877272.86 |4402 |9662.0 |
2013-11 |CLOSED |339567.75 |1693 |3725.0 |
2013-11 |COMPLETE |1040367.58 |5201 |11400.0 |
2013-12 |CLOSED |348057.14 |1732 |3727.0 |
2013-12 |COMPLETE |929662.46 |4646 |10055.0 |
2014-01 |CLOSED |298236.03 |1492 |3247.0 |
2014-01 |COMPLETE |931985.71 |4706 |9958.0 |
2014-02 |CLOSED |302636.05 |1547 |3251.0 |
2014-02 |COMPLETE |915134.04 |4667 |10251.0 |
2014-03 |CLOSED |301673.83 |1514 |3344.0 |
2014-03 |COMPLETE |969677.14 |4880 |10642.0 |
2014-04 |CLOSED |297959.38 |1463 |3133.0 |
2014-04 |COMPLETE |951764.14 |4838 |10600.0 |
2014-05 |CLOSED |294133.04 |1468 |3212.0 |
2014-05 |COMPLETE |927546.29 |4665 |10165.0 |
2014-06 |CLOSED |270021.59 |1341 |2966.0 |
2014-06 |COMPLETE |909732.47 |4465 |9938.0 |
2014-07 |CLOSED |258298.87 |1269 |2769.0 |
2014-07 |COMPLETE |697291.9 |3568 |7703.0 |
+----------+------------+--------------------+-----------------------+-----------------------+